In [258]:
import pymysql
import pymysql.cursors
import re
import datetime


ts = datetime.datetime.now().strftime("%a%d%b%Y-%I-%M-%S%p")
print ("Current TimeStamp: ",ts)

Current TimeStamp:  Wed23Jun2021-04-50-18AM


In [259]:
db = pymysql.connect(host='localhost',user='root',password='',database='material-db',cursorclass=pymysql.cursors.DictCursor)
cur = db.cursor()

cur.execute("Select id,papername,doi,clnsent,token,label from datatable")
result_list = cur.fetchall()

print(type(result_list))
print(len(result_list))


# db.commit()
# db.close()

<class 'list'>
42228


In [264]:
def clean_text(text):
    text = re.sub(r"(^[\s])","",text) #removing starting space of any sentence   (^[\d])
    text = re.sub(r"(Ϫ)","-",text) # replacing this character Ϫ with -
    text = re.sub(r"(^[0-9]+)","",text) #removing starting number of any sentence   (^[\d])
    text = re.sub(r"(\\.)",".",text) # replacing \. with .
    text = re.sub(r"(-)"," ",text) # replacing - with <space>
    text = re.sub(r"(\d)(°C)",r"\1 \2",text) # separating °C sign from numbers
    text = re.sub(r"(\d)([hH])",r"\1 \2",text) # separating Hh from number
    text = re.sub(r"(\d)([mM])",r"\1 \2",text) # separating mM from number
    text = re.sub(r"(˚C)","°C",text) # replacing °C sign properly
    text = re.sub(r"(º C)","°C",text) # replacing °C sin properly
    text = re.sub(r"[∼~]","",text)
    text = re.sub(r"[`']","",text)
    text = re.sub(r"(^\s[0-9]+)","",text) # replacing <space>Number from begining of the sent created by replacing Ϫ with -
   
    return text



#https://stackoverflow.com/questions/354038/how-do-i-check-if-a-string-is-a-number-float?rq=1
def is_number(s):
    try:
        float(s) # for int, long and float
    except ValueError:
        try:
            complex(s) # for complex
        except ValueError:
            return False

    return True

#https://stackoverflow.com/questions/17686809/how-to-find-word-next-to-a-word-in-python
def nextword(target, source):
    for i, w in enumerate(source):
        if w == target:
            if i<len(source)-1:
                return source[i+1]
            else: 
                return "NULL-unit"


In [268]:
ts = datetime.datetime.now().strftime("%a%d%b%Y-%I-%M-%S%p")
counter = 0
ntwriter = open("material-kg-rdf-"+ts+".nt","a+",encoding="utf-8")
type_def = '''<http://ontology.ontotext.com/document> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/document> .
<http://ontology.ontotext.com/paper> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/paper> .
<http://ontology.ontotext.com/paper> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/document> .
<http://ontology.ontotext.com/doi> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/doi> .
<http://ontology.ontotext.com/label> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/label> .
<http://ontology.ontotext.com/token> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/token> .
<http://ontology.ontotext.com/sentence> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/sentence> .
<http://ontology.ontotext.com/value> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/value> .
'''
ntwriter.write(type_def)
ntwriter.write("\n")

for item in result_list:
    counter +=1
    # strtst = result_list[0] # 345  8743 8978  68 73 35 56
    strtst = item
    cleaned_sent = clean_text(strtst["clnsent"])
    cleaned_sent_rdf = re.sub(r"(\s)","-",cleaned_sent)
    cleaned_sent_rdf = re.sub(r"([<>])","-",cleaned_sent_rdf)
    cleaned_sent_rdf = re.sub(r"(%)","PCTG",cleaned_sent_rdf) # Need to remove | # space [ ] ' " , and many more also
    sent_token = strtst["token"]
    sent_token_rdf = re.sub(r"(\s)","-",sent_token)
    sent_label = strtst["label"]
    sent_label_rdf = re.sub(r"(\s)","-",sent_label)
    paper_name = strtst["papername"]
    paper_name_rdf = re.sub(r"(\s)","-",paper_name)
    paper_name_rdf = re.sub(r"[\[\]\'\"\,]","",paper_name_rdf)
    paper_doi = strtst["doi"]
    sent_id = strtst["id"]

    ntwriter.write("<http://ontology.ontotext.com/"+paper_name_rdf+"> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/paper> ." + "\n")
    ntwriter.write("<http://ontology.ontotext.com/"+paper_name_rdf+"> <http://ontology.ontotext.com#Is_A> <http://ontology.ontotext.com/paper> ." + "\n")  #need to check this entry after visualization
    ntwriter.write("<http://ontology.ontotext.com/"+paper_name_rdf+"> <http://ontology.ontotext.com#Is_A> <http://ontology.ontotext.com/document> ." + "\n")
    ntwriter.write("<http://ontology.ontotext.com/"+paper_doi+"> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/doi> ." + "\n")
    ntwriter.write("<http://ontology.ontotext.com/"+paper_doi+"> <http://ontology.ontotext.com#Is_A> <http://ontology.ontotext.com/doi> ." + "\n")
    ntwriter.write("<http://ontology.ontotext.com/"+paper_name_rdf+"> <http://ontology.ontotext.com#Contains> <http://ontology.ontotext.com/"+paper_doi+"> ." + "\n")
    ntwriter.write("<http://ontology.ontotext.com/"+cleaned_sent_rdf+"> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/sentence> ." + "\n")
    ntwriter.write("<http://ontology.ontotext.com/"+cleaned_sent_rdf+"> <http://ontology.ontotext.com#Is_A> <http://ontology.ontotext.com/sentence> ." + "\n")
    ntwriter.write("<http://ontology.ontotext.com/"+paper_name+"> <http://ontology.ontotext.com#Contains> <http://ontology.ontotext.com/"+cleaned_sent_rdf+"> ." + "\n")
    ntwriter.write("<http://ontology.ontotext.com/"+paper_doi+"> <http://ontology.ontotext.com#Contains> <http://ontology.ontotext.com/"+cleaned_sent_rdf+"> ." + "\n")
    ntwriter.write("<http://ontology.ontotext.com/"+sent_token_rdf+"> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/token> ." + "\n")
    ntwriter.write("<http://ontology.ontotext.com/"+sent_token_rdf+"> <http://ontology.ontotext.com#Is_A> <http://ontology.ontotext.com/token> ." + "\n")
    ntwriter.write("<http://ontology.ontotext.com/"+cleaned_sent_rdf+"> <http://ontology.ontotext.com#Contains> <http://ontology.ontotext.com/"+sent_token_rdf+"> ." + "\n")
    ntwriter.write("<http://ontology.ontotext.com/"+sent_label_rdf+"> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/label> ." + "\n")
    ntwriter.write("<http://ontology.ontotext.com/"+sent_label_rdf+"> <http://ontology.ontotext.com#Is_A> <http://ontology.ontotext.com/label> ." + "\n")
    ntwriter.write("<http://ontology.ontotext.com/"+cleaned_sent_rdf+"> <http://ontology.ontotext.com#Contains> <http://ontology.ontotext.com/"+sent_label_rdf+"> ." + "\n")
    ntwriter.write("<http://ontology.ontotext.com/"+sent_token_rdf+"> <http://ontology.ontotext.com#Is> <http://ontology.ontotext.com/"+sent_label_rdf+"> ." + "\n")
    ntwriter.write("<http://ontology.ontotext.com/"+paper_name_rdf+"> <http://ontology.ontotext.com#Contains> <http://ontology.ontotext.com/"+sent_token_rdf+"> ." + "\n")
    ntwriter.write("<http://ontology.ontotext.com/"+paper_name_rdf+"> <http://ontology.ontotext.com#Contains> <http://ontology.ontotext.com/"+sent_label_rdf+"> ." + "\n")
    ntwriter.write("<http://ontology.ontotext.com/"+paper_doi+"> <http://ontology.ontotext.com#Contains> <http://ontology.ontotext.com/"+sent_token_rdf+"> ." + "\n")
    ntwriter.write("<http://ontology.ontotext.com/"+paper_doi+"> <http://ontology.ontotext.com#Contains> <http://ontology.ontotext.com/"+sent_label_rdf+"> ." + "\n")

    sent_token_position = cleaned_sent.find(sent_token)
    all_toks_sent = cleaned_sent.split(" ")
    # all_numbers = re.findall(r'([-+]?[0-9]*\.?[0-9]+)',cleaned_sent) # find all the numbers in this string

    all_numbers = [x for x in all_toks_sent if is_number(x)]
    if (len(all_numbers))>0:
            max_pos = cleaned_sent.find(all_numbers[-1])
            min_pos = cleaned_sent.find(all_numbers[0])
            max_pos_value = None # "NULL" -- for db ops #all_numbers[-1]
            min_pos_value = None # "NULL" -- for db ops #all_numbers[0]
            max_pos_value_unit = None # "NULL" -- for db ops 
            min_pos_value_unit = None # "NULL" -- for db ops 
            
            for number in all_numbers:
                current_pos = cleaned_sent.find(number)
                if current_pos == sent_token_position or current_pos > sent_token_position:
                    max_pos = current_pos
                    max_pos_value = number
                elif current_pos < sent_token_position:
                    min_pos = current_pos
                    min_pos_value = number
                # elif current_pos>sent_token_position and current_pos<max_pos:
                #     max_pos = current_pos
                #     max_pos_value = number
            # print(cleaned_sent)
            # print ("Token: ",sent_token,"token pos: ",sent_token_position)
            # print("left: ",sent_token,":",min_pos_value)
            # print("right: ",sent_token,":",max_pos_value)
            if max_pos_value is not None:
                max_pos_value_unit = nextword(max_pos_value,all_toks_sent)
                max_pos_value_unit_rdf = re.sub(r"(%)","PCTG",max_pos_value_unit)
            if min_pos_value is not None:
                min_pos_value_unit = nextword(min_pos_value,all_toks_sent)
                min_pos_value_unit_rdf = re.sub(r"(%)","PCTG",min_pos_value_unit)
            
            if min_pos_value_unit is not None:
                ntwriter.write("<http://ontology.ontotext.com/"+sent_token_rdf+"> <http://ontology.ontotext.com#HasLvalue> <http://ontology.ontotext.com/"+min_pos_value+"-"+min_pos_value_unit_rdf+"> ." + "\n")
            if max_pos_value_unit is not None:
                ntwriter.write("<http://ontology.ontotext.com/"+sent_token_rdf+"> <http://ontology.ontotext.com#HasRvalue> <http://ontology.ontotext.com/"+max_pos_value+"-"+max_pos_value_unit_rdf+"> ." + "\n")
            print(paper_name," : Processed!")
            # print ("Token: ",sent_token,"token pos: ",sent_token_position)
            # print("left: ",sent_token,":",min_pos_value,"-->",min_pos_value_unit)
            # print("right: ",sent_token,":",max_pos_value,"-->",max_pos_value_unit)
            # cur1 = db.cursor()
            # cur1.execute("UPDATE `datatable1clone` SET clnsent=%s,lvalue=%s,lunit=%s,rvalue=%s,runit=%s where id=%s", (cleaned_sent,min_pos_value,min_pos_value_unit,max_pos_value,max_pos_value_unit,sent_id))
            # db.commit()
    else:
        ntwriter.write("<http://ontology.ontotext.com/"+sent_token_rdf+"> <http://ontology.ontotext.com#IsNull> <http://ontology.ontotext.com/NullValue> ." + "\n")
        print(paper_name," : Processed!")
    print("Sentence No: ",counter)
        # print(sent_token)
        # print("no value found")
        # cur1 = db.cursor()
        # cur1.execute("UPDATE `datatable1clone` SET clnsent=%s,lvalue=%s,lunit=%s,rvalue=%s,runit=%s where id=%s", (cleaned_sent,"NULL","NULL","NULL","NULL",sent_id))
        # db.commit()
    # print(all_numbers)
    # for number in all_numbers:
    #     pos = cleaned_sent.find(number)
    #     print(number,pos)

ntwriter.close()

SyntaxError: invalid syntax (<ipython-input-268-5384213ebae9>, line 30)

Experimental cells

In [ ]:
strtst = result_list[8743]

print (type(strtst))
for t,l in strtst.items():
    cleaned_sent = ""
    sent_token = ""
    if t == "clnsent":
        cleaned_sent = clean_text(l)
    if t == "token":
        sent_token = clean_text(l)
    print(cleaned_sent,"==>",sent_token)

In [ ]:
strtst = result_list[345]
cleaned_sent = clean_text(strtst["clnsent"])
sent_token = strtst["token"]
all_toks_sent = cleaned_sent.split(" ")
# print (all_toks_sent)
all_numbers = re.findall(r'([-+]?[0-9]*\.?[0-9]+)',cleaned_sent)
print(len(all_numbers))
# pattern = re.compile(r'([-+]?[0-9]*\.?[0-9]+)')
# for m in re.finditer(pattern, cleaned_sent):
#     print (m.group(),":",m.start())

for number in all_numbers:
    pos = cleaned_sent.find(number)
    print(number,pos)
    # print(number,"==>",re.search(number,cleaned_sent))

max_pos = cleaned_sent.find(all_numbers[-1])
min_pos = cleaned_sent.find(all_numbers[0])

print(cleaned_sent,"==>",sent_token)
print(all_numbers)
print(max_pos,min_pos)
print(all_numbers[0])
# print (type(strtst))
# for t,l in strtst.items():
#     cleaned_sent = ""
#     sent_token = ""
#     if t == "clnsent":
#         cleaned_sent = clean_text(l)
#     if t == "token":
#         sent_token = clean_text(l)
#     print(cleaned_sent,"==>",sent_token)

In [ ]:
s="12Ϫ15Until recently temperaturedependent studies supercapacitors assembled conventional carbonaceous materials organic electrolyte limited 70°C"

pattern = re.compile(r'([0-9]+.*)([A-Z]+)')

for m in re.finditer(pattern, s):
    print (m.group(2), '*', m.group(1))

In [ ]:
i = 0
for item in result_list[370]:
    strtst = item
    cleaned_sent = clean_text(strtst["clnsent"])
    sent_token = strtst["token"]
    print(sent_token)
    print(cleaned_sent)

In [ ]:
def is_number(s):
    try:
        float(s) # for int, long and float
    except ValueError:
        try:
            complex(s) # for complex
        except ValueError:
            return False

    return True
    
print(is_number("10000s"))

In [ ]:
strtst = result_list[8978]
cleaned_sent = clean_text(strtst["clnsent"])
sent_token = strtst["token"]
print(sent_token)
print(cleaned_sent)
all_toks_sent = cleaned_sent.split(" ")
all_numbers = [x for x in all_toks_sent if is_number(x)]

print (all_numbers)

In [ ]:
############### Code backup ###################
# for item in
strtst = result_list[8978] # 345  8743
cleaned_sent = clean_text(strtst["clnsent"])
sent_token = strtst["token"]

sent_token_position = cleaned_sent.find(sent_token)
all_toks_sent = cleaned_sent.split(" ")
# all_numbers = re.findall(r'([-+]?[0-9]*\.?[0-9]+)',cleaned_sent) # find all the numbers in this string

all_numbers = [x for x in all_toks_sent if is_number(x)]
if (len(all_numbers))>0:
        max_pos = cleaned_sent.find(all_numbers[0])
        min_pos = cleaned_sent.find(all_numbers[0])
        max_pos_value = 0  #all_numbers[-1]
        min_pos_value = 0  #all_numbers[0]
        for number in all_numbers:
            current_pos = cleaned_sent.find(number)
            if current_pos < sent_token_position:
                min_pos = current_pos
                min_pos_value = number
            elif current_pos>sent_token_position and current_pos<max_pos:
                max_pos = current_pos
                max_pos_value = number

            print(cleaned_sent)
            print ("token pos: ",sent_token_position)
            print("left: ",sent_token,":",min_pos_value)
            print("right: ",sent_token,":",max_pos_value)
else:
    print(cleaned_sent)
    print("no value found")


In [ ]:
############ Code backup db-version with left right value and units in datatable1clone ##############
# db.close()
db1 = pymysql.connect(host='localhost',user='root',password='',database='material-db',cursorclass=pymysql.cursors.DictCursor)
for item in result_list:
    # strtst = result_list[0] # 345  8743 8978  68 73 35 56
    strtst = item
    cleaned_sent = clean_text(strtst["clnsent"])
    sent_token = strtst["token"]
    sent_label = strtst["label"]
    paper_name = strtst["papername"]
    paper_doi = strtst["doi"]
    sent_id = strtst["id"]

    sent_token_position = cleaned_sent.find(sent_token)
    all_toks_sent = cleaned_sent.split(" ")
    # all_numbers = re.findall(r'([-+]?[0-9]*\.?[0-9]+)',cleaned_sent) # find all the numbers in this string

    all_numbers = [x for x in all_toks_sent if is_number(x)]
    if (len(all_numbers))>0:
            max_pos = cleaned_sent.find(all_numbers[-1])
            min_pos = cleaned_sent.find(all_numbers[0])
            max_pos_value = "NULL"  #all_numbers[-1]
            min_pos_value = "NULL"  #all_numbers[0]
            max_pos_value_unit = "NULL"
            min_pos_value_unit = "NULL"
            
            for number in all_numbers:
                current_pos = cleaned_sent.find(number)
                if current_pos == sent_token_position or current_pos > sent_token_position:
                    max_pos = current_pos
                    max_pos_value = number
                elif current_pos < sent_token_position:
                    min_pos = current_pos
                    min_pos_value = number
                # elif current_pos>sent_token_position and current_pos<max_pos:
                #     max_pos = current_pos
                #     max_pos_value = number
            # print(cleaned_sent)
            # print ("Token: ",sent_token,"token pos: ",sent_token_position)
            # print("left: ",sent_token,":",min_pos_value)
            # print("right: ",sent_token,":",max_pos_value)
            if max_pos_value != "NULL":
                max_pos_value_unit = nextword(max_pos_value,all_toks_sent)
            if min_pos_value != "NULL":
                min_pos_value_unit = nextword(min_pos_value,all_toks_sent)
            print(cleaned_sent)
            print ("Token: ",sent_token,"token pos: ",sent_token_position)
            print("left: ",sent_token,":",min_pos_value,"-->",min_pos_value_unit)
            print("right: ",sent_token,":",max_pos_value,"-->",max_pos_value_unit)
            cur2 = db1.cursor()
            cur2.execute("UPDATE `datatable1clone` SET clnsent=%s,lvalue=%s,lunit=%s,rvalue=%s,runit=%s where id=%s", (cleaned_sent,min_pos_value,min_pos_value_unit,max_pos_value,max_pos_value_unit,sent_id))
            db1.commit()
    else:
        print(cleaned_sent)
        print(sent_token)
        print("no value found")
        cur2 = db1.cursor()
        cur2.execute("UPDATE `datatable1clone` SET clnsent=%s,lvalue=%s,lunit=%s,rvalue=%s,runit=%s where id=%s", (cleaned_sent,"NULL","NULL","NULL","NULL",sent_id))
        db1.commit()
    # print(all_numbers)
    # for number in all_numbers:
    #     pos = cleaned_sent.find(number)
    #     print(number,pos)



In [ ]:
ntwriter = open("material-kg-rdf-"+ts+".nt","a+",encoding="utf-8")
type_def = '''<http://ontology.ontotext.com/document> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/document> .
<http://ontology.ontotext.com/paper> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/document> .
<http://ontology.ontotext.com/doi> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/doi> .
<http://ontology.ontotext.com/label> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/label> .
<http://ontology.ontotext.com/token> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/token> .
<http://ontology.ontotext.com/sentence> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/sentence> .
<http://ontology.ontotext.com/value> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://ontology.ontotext.com/value> .
'''
ntwriter.write(type_def)
ntwriter.close()